## Definitions

### Observables
+ time <span style="background-color: #F5F5F5">$t_{k}$</span>
+ position <span style="background-color: #F5F5F5">$m_{k}$</span>
+ velocity <span style="background-color: #F5F5F5">$v_{k}$</span>
+ LFP power <span style="background-color: #F5F5F5">$y_{k}$</span>
+ population spiking activity of $C$ cells <span style="background-color: #F5F5F5">$\Delta N_{k}^{1:C}$</span>


### (Semi-)Latent Variables
+ replay indicator
$
  \begin{equation}
    I_{k} =
    \begin{cases}
      0, & \text{if replay at}\ t_{k} \\
      1, & \text{if no replay at}\ t_{k} \\
    \end{cases}
  \end{equation}
$
+ position/replay position 
$
  \begin{equation}
    x_{k} =
    \begin{cases}
      \text{unobserved replay position}, & \text{if}\ I_{k} = 1 \\
      m_{k}, & \text{if}\ I_{k} = 0 \\
    \end{cases}
  \end{equation}
$


## Filter

### State Transitions
+ replay state transition $\Pr(I_{k} \mid I_{k-1}, v_{k-1})$
+ movement state transition 
$
  \begin{equation}
    p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) =
    \begin{cases}
      \delta (m_{k}), & \text{if}\ I_{k} = 0 \\
      U(a, b), & \text{if}\ I_{k} = 1 \text{ and }\ I_{k-1} = 0 \\
      N(x_{k-1}, \sigma), & \text{if}\ I_{k} = 1 \text{ and }\ I_{k-1} = 1 \\
    \end{cases}
  \end{equation}
$

### Likelihoods
+ velocity likelihood $p(v_{k} \mid I_{k}, v_{k})$
+ LFP power likelihood $p(y_{k} \mid I_{k}, y_{k})$
+ population spiking activity likelihood $p(\Delta N_{k}^{1:C} \mid I_{k}, )$

### Joint Filter Distribution
+ $p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})$


### Calculation Goals
+ replay probability $\Pr(I_{k} = 1 \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})$
+ replay position $p(x_{k}, I_{k} = 1\mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})$

## Filter Derivation


1. Bayes ($ posterior \propto likelihood * prior $)
$$
p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k}) \propto
p(y_{k} \mid I_{k}, x_{k}, y_{1:k-1}, \Delta N_{1:k}^{1:C}, v_{1:k}) *
p(\Delta N_{k}^{1:C} \mid I_{k}, x_{k}, y_{1:k}, \Delta N_{1:k-1}^{1:C}, v_{1:k}) * 
p(v_{k} \mid I_{k}, x_{k}, y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) *
p(I_{k}, x_{k} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1})
$$

2. Simplify likelihoods
$$
p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})  \propto 
p(y_{k} \mid I_{k}, y_{1:k-1}) *
p(\Delta N_{k}^{1:C} \mid x_{k}, \Delta N_{1:k-1}^{1:C}) *
p(v_{k} \mid I_{k}, v_{1:k-1}) *
p(I_{k}, x_{k} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1})
$$

3. Chapman-Kolmogorov to define prior in terms of previous time step's posterior
$$
\begin{align*}
p(I_{k}, x_{k} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) &= \sum_{I_{k-1}} \int p(I_{k}, I_{k-1}, x_{k}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
\\ &= \sum_{I_{k-1}} \int p(I_{k}, x_{k} \mid I_{k-1}, x_{k-1}, y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \tag{Markov}
\\ &= \sum_{I_{k-1}} \int p(I_{k}, x_{k} \mid I_{k-1}, x_{k-1}) * p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \tag{Markov}
\\ &= \sum_{I_{k-1}} \int p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) * \Pr(I_{k} \mid I_{k-1}, v_{k-1}) *
p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
\end{align*}$$

4. Final Filter
$$
p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})  \propto 
p(y_{k} \mid I_{k}, y_{1:k-1}) *
p(\Delta N_{k}^{1:C} \mid x_{k}, \Delta N_{1:k-1}^{1:C}) *
p(v_{k} \mid I_{k}, v_{1:k-1}) *
\sum_{I_{k-1}} \int p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) * \Pr(I_{k} \mid I_{k-1}, v_{k-1}) *
p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
$$

## Computing goals from filter

+ replay position
$$
\begin{align*}
p(x_{k} \mid I_{k} = 1, y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k}) &= \frac{p(x_{k}, I_{k} = 1 \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})}{\Pr(I_{k} = 1 \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})}  \tag{Def. of conditional probability}
\\
\\ p(x_{k}, I_{k} = 1 \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k}) &\propto p(y_{k} \mid I_{k} = 1, y_{1:k-1}) *
p(\Delta N_{k}^{1:C} \mid x_{k}, \Delta N_{1:k-1}^{1:C}) *
p(v_{k} \mid I_{k} = 1, v_{1:k-1}) *
\sum_{I_{k-1}} \int p(x_{k} \mid x_{k-1}, I_{k} = 1, I_{k-1}) * \Pr(I_{k} = 1 \mid I_{k-1}, v_{k-1}) *
p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
\\ &\propto \text{likelihoods} * \Bigg(
\int p(x_{k} \mid x_{k-1}, I_{k} = 1, I_{k-1} = 0) * \Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) *
p(I_{k-1} = 0, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} +
\int p(x_{k} \mid x_{k-1}, I_{k} = 1, I_{k-1} = 1) * \Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) *
p(I_{k-1} = 1, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \Bigg)
\\ &\propto \text{likelihoods} * \Bigg( \Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * \int U(a, b) * 
p(I_{k-1} = 0, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} +
\Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) * \int N(x_{k-1}, \sigma) * 
p(I_{k-1} = 1, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \Bigg)
\end{align*}$$

### Computation

#### Initialization

+ $Pr(I_{0} = 0) = 1$ Assume we are not in a replay at the first timestep
+ $p(x_{0}, I_{0} = 0) = \delta(m_{0}) $


#### Predict Step
Predict prior = $p(x_{k}, I_{k} \mid H_{k-1}) = \sum_{I_{k-1}} \int p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) * \Pr(I_{k} \mid I_{k-1}, v_{k-1}) * p(x_{k-1}, I_{k-1}) dx_{k-1}$

+ $I_{k + 1} = 0, I_{k} = 0$
$$
\begin{align*}
p(x_{k}, I_{k} = 0, I_{k-1} = 0 \mid H_{1:k-1}) &= \int p(x_{k} \mid x_{k-1}, I_{k-1} = 0, I_{k} = 0) * Pr(I_{k} = 0 \mid I_{k-1} = 0, v_{k-1}) * p(x_{k-1}, I_{k-1} = 0 \mid H_{1:k-1}) dx_{k-1}
\\ &= \int \delta(m_{k}) * Pr(I_{k} = 0 \mid I_{k-1} = 0, v_{k-1}) * p(x_{k-1}, I_{k-1} = 0 \mid H_{1:k-1}) dx_{k-1}
\\ &= \delta(m_{k}) * Pr(I_{k} = 0 \mid I_{k-1} = 0, v_{k-1}) * \int  p(x_{k-1}, I_{k-1} = 0 \mid H_{1:k-1}) dx_{k-1}
\\ &= \delta(m_{k}) * Pr(I_{k} = 0 \mid I_{k-1} = 0, v_{k-1}) * Pr(I_{k-1} = 0)
\\ &= \delta(m_{k}) * \big(1 - Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1})\big) * Pr(I_{k-1} = 0)
\end{align*}
$$

+ $I_{k + 1} = 0, I_{k} = 1$
$$
\begin{align*}
p(x_{k}, I_{k} = 0, I_{k-1} = 1 \mid H_{1:k-1}) &= \int p(x_{k} \mid x_{k-1}, I_{k-1} = 1, I_{k} = 0) * Pr(I_{k} = 0 \mid I_{k-1} = 1, v_{k-1}) * p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
\\ &= \int \delta(m_{k}) * Pr(I_{k} = 0 \mid I_{k-1} = 1, v_{k-1}) * p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
\\ &= \delta(m_{k}) * Pr(I_{k} = 0 \mid I_{k-1} = 1, v_{k-1}) * \int  p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
\\ &= \delta(m_{k}) * Pr(I_{k} = 0 \mid I_{k-1} = 1, v_{k-1}) * Pr(I_{k-1} = 1)
\\ &= \delta(m_{k}) * \big(1 - Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1})\big) * Pr(I_{k-1} = 1)
\end{align*}
$$

+ $I_{k} = 1, I_{k-1} = 0$
$$
\begin{align*}
p(x_{k}, I_{k} = 1, I_{k-1} = 0 \mid H_{1:k-1}) &= \int p(x_{k} \mid x_{k-1}, I_{k-1} = 0, I_{k} = 1) * Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * p(x_{k-1}, I_{k-1} = 0 \mid H_{1:k-1}) dx_{k-1}
\\ &= \int \mathcal{U}(a, b) * Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * p(x_{k-1}, I_{k-1} = 0 \mid H_{1:k-1}) dx_{k-1}
\\ &= \mathcal{U}(a, b) * Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * \int p(x_{k-1}, I_{k-1} = 0 \mid H_{1:k-1}) dx_{k-1}
\\ &= \mathcal{U}(a, b) * Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * Pr(I_{k-1} = 0)
\end{align*}
$$

+ $I_{k} = 1, I_{k-1} = 1$
$$
\begin{align*}
p(x_{k}, I_{k} = 1, I_{k-1} = 1 \mid H_{1:k-1}) &= \int p(x_{k} \mid x_{k-1}, I_{k-1} = 1, I_{k} = 1) * Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) * p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
\\ &= \int \mathcal{N}(x_{k-1}, \hat{\sigma}) * Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) * p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
\\ &= Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) * \int \mathcal{N}(x_{k-1}, \hat{\sigma}) * p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
\end{align*}
$$


So we have:

$$
p(x_{k}, I_{k} = 0 \mid H_{1:k-1}) = \delta(m_{k}) * \big[\big(1 - Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1})\big) * Pr(I_{k-1} = 0) + \big(1 - Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1})\big) * Pr(I_{k-1} = 1)\big]
$$

and

$$
p(x_{k}, I_{k} = 1 \mid H_{1:k-1}) = \mathcal{U}(a, b) * Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * Pr(I_{k-1} = 0) + Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) * \int \mathcal{N}(x_{k-1}, \hat{\sigma}) * p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
$$

### Update Step

$$
p(x_{k}, I_{k} \mid H_{1:k}) = Normalize\Big(p(x_{k}, I_{k} \mid H_{k-1}) * likelihoods_{k}\Big)
$$

### Algorithm

Set initial conditions:
+ $Pr(I_{0} = 0) = 1$
+ $p(x_{0}, I_{0} = 0) = \delta(m_{0}) $

For each time step $k$:
1. Predict prior
    $$
    p(x_{k}, I_{k} = 0 \mid H_{1:k-1}) = \delta(m_{k}) * \big[\big(1 - Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1})\big) * Pr(I_{k-1} = 0) + \big(1 - Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1})\big) * Pr(I_{k-1} = 1)\big]
    $$

    and

    $$
    p(x_{k}, I_{k} = 1 \mid H_{1:k-1}) = \mathcal{U}(a, b) * Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * Pr(I_{k-1} = 0) + Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) * \int \mathcal{N}(x_{k-1}, \hat{\sigma}) * p(x_{k-1}, I_{k-1} = 1 \mid H_{1:k-1}) dx_{k-1}
    $$
    
2. Update Posterior

$$
p(x_{k}, I_{k} \mid H_{1:k}) = Normalize\Big(p(x_{k}, I_{k} \mid H_{k-1}) * likelihoods_{k}\Big)
$$

## Smoother

### Smoother State Transitions

+ Movement State Transition $p(x_{k+1} \mid x_{k}, I_{k}, I_{k+1})$

For the filter we fit:
$$
 \begin{equation}
    p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) = 
    \begin{cases}
      \delta(m_{k}), & \text{if}\ I_{k} = 0, I_{k-1} \in \{0, 1\} \\
      \mathcal{U}(a, b), & \text{if}\ I_{k} = 1, I_{k-1} = 0 \\
      \mathcal{N}(x_{k-1}, \hat{\sigma}), & \text{if}\ I_{k} = 1, I_{k-1} = 1 \\
    \end{cases}
  \end{equation}
$$

So

$$
 \begin{equation}
    p(x_{k + 1} \mid x_{k}, I_{k+1}, I_{k}) = 
    \begin{cases}
      \delta(m_{k+1}), & \text{if}\ I_{k+1} = 0, I_{k} \in \{0, 1\} \\
      \mathcal{U}(a, b), & \text{if}\ I_{k+1} = 1, I_{k} = 0 \\
      \mathcal{N}(x_{k}, \hat{\sigma}), & \text{if}\ I_{k+1} = 1, I_{k} = 1 \\
    \end{cases}
  \end{equation}
$$

+ Replay State Transition $Pr(I_{k + 1} \mid I_{k}, v_{k})$

For the filter, we fit $Pr(I_{k} \mid I_{k-1}, v_{k-1})$, so we can just use it evaluated at $k+1$

### Predict Step
Predict the prior distribution $p(x_{k + 1}, I_{k + 1} \mid H_{1:k})$ using the filter distribution $p(x_{k}, I_{k} \mid H_{1:k})$ and state transition $p(x_{k+1} \mid x_{k}, I_{k}, I_{k+1}) * Pr(I_{k + 1} \mid I_{k}, v_{k}) $:
$$prior = p(x_{k + 1}, I_{k + 1} \mid H_{1:k}) = \sum_{I_{k}} \int p(x_{k+1} \mid x_{k}, I_{k}, I_{k+1}) * Pr(I_{k + 1} \mid I_{k}, v_{k}) * p(x_{k}, I_{k} \mid H_{1:k}) dx_{k}
$$

+ $I_{k + 1} = 0, I_{k} = 0$
$$
\begin{align*}
p(x_{k + 1}, I_{k + 1} = 0, I_{k} = 0 \mid H_{1:k}) &= \int p(x_{k+1} \mid x_{k}, I_{k} = 0, I_{k+1} = 0) * Pr(I_{k + 1} = 0 \mid I_{k} = 0, v_{k}) * p(x_{k}, I_{k} = 0 \mid H_{1:k}) dx_{k}
\\ &= \int \delta(m_{k+1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 0, v_{k}) * p(x_{k}, I_{k} = 0 \mid H_{1:k}) dx_{k}
\\ &= \delta(m_{k+1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 0, v_{k}) * \int  p(x_{k}, I_{k} = 0 \mid H_{1:k}) dx_{k}
\\ &= \delta(m_{k+1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 0, v_{k}) * Pr(I_{k} = 0)
\\ &= \delta(m_{k+1}) * \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k})\big) * Pr(I_{k} = 0)
\end{align*}
$$

+ $I_{k + 1} = 0, I_{k} = 1$
$$
\begin{align*}
p(x_{k + 1}, I_{k + 1} = 0, I_{k} = 1 \mid H_{1:k}) &= \int p(x_{k+1} \mid x_{k}, I_{k} = 1, I_{k+1} = 0) * Pr(I_{k + 1} = 0 \mid I_{k} = 1, v_{k}) * p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
\\ &= \int \delta(m_{k+1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 1, v_{k}) * p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
\\ &= \delta(m_{k+1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 1, v_{k}) * \int  p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
\\ &= \delta(m_{k+1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 1, v_{k}) * Pr(I_{k} = 1)
\\ &= \delta(m_{k+1}) * \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k})\big) * Pr(I_{k} = 1)
\end{align*}
$$

+ $I_{k + 1} = 1, I_{k} = 0$
$$
\begin{align*}
p(x_{k + 1}, I_{k + 1} = 1, I_{k} = 0 \mid H_{1:k}) &= \int p(x_{k+1} \mid x_{k}, I_{k} = 0, I_{k+1} = 1) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * p(x_{k}, I_{k} = 0 \mid H_{1:k}) dx_{k}
\\ &= \int \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * p(x_{k}, I_{k} = 0 \mid H_{1:k}) dx_{k}
\\ &= \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * \int p(x_{k}, I_{k} = 0 \mid H_{1:k}) dx_{k}
\\ &= \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * Pr(I_{k} = 0)
\end{align*}
$$

+ $I_{k + 1} = 1, I_{k} = 1$
$$
\begin{align*}
p(x_{k + 1}, I_{k + 1} = 1, I_{k} = 1 \mid H_{1:k}) &= \int p(x_{k+1} \mid x_{k}, I_{k} = 1, I_{k+1} = 1) * Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
\\ &= \int \mathcal{N}(x_{k}, \hat{\sigma}) * Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
\\ &= Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * \int \mathcal{N}(x_{k}, \hat{\sigma}) * p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
\end{align*}
$$

So we have:

$$
p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k}) = \delta(m_{k+1}) * \big[\big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k})\big) * Pr(I_{k} = 0) + \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k})\big) * Pr(I_{k} = 1)\big]
$$

and

$$
p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k}) = \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * Pr(I_{k} = 0) + Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * \int \mathcal{N}(x_{k}, \hat{\sigma}) * p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
$$

## Backwards Update Step

Update the smoother distribution $p(x_{k}, I_{k} \mid H_{1:T})$ using the filter distribution $p(x_{k}, I_{k} \mid H_{1:k})$, prior $p(x_{k + 1}, I_{k + 1} \mid H_{1:k})$ and state transition $p(x_{k+1} \mid x_{k}, I_{k}, I_{k+1}) * Pr(I_{k + 1} \mid I_{k}, v_{k}) $:
$$
p(x_{k}, I_{k} \mid H_{1:T}) = p(x_{k}, I_{k} \mid H_{1:k}) * weights
$$

where:
$$
weights = \sum_{I_{k+1}} \int \Big[ \frac{p(x_{k+1} \mid x_{k}, I_{k}, I_{k+1}) * Pr(I_{k + 1} \mid I_{k}, v_{k}) * p(x_{k+1}, I_{k+1} \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} \mid H_{1:k})} \Big] dx_{k+1}
$$


+ $I_{k + 1} = 0, I_{k} = 0$
$$
\begin{align*}
weights &= \int \Big[ \frac{p(x_{k+1} \mid x_{k}, I_{k} = 0, I_{k+1} = 0) * Pr(I_{k + 1} = 0 \mid I_{k} = 0, v_{k}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \int \Big[ \frac{\delta(m_{k + 1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 0, v_{k}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= Pr(I_{k + 1} = 0 \mid I_{k} = 0, v_{k}) * \int \Big[ \frac{\delta(m_{k + 1}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}\big) * \int \Big[ \frac{\delta(m_{k + 1}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}\big) * \frac{p(x_{k+1}=m_{k + 1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}=m_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})}
\end{align*}
$$

+ $I_{k + 1} = 0, I_{k} = 1$
$$
\begin{align*}
weights &= \int \Big[ \frac{p(x_{k+1} \mid x_{k}, I_{k} = 1, I_{k+1} = 0) * Pr(I_{k + 1} = 0 \mid I_{k} = 1, v_{k}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \int \Big[ \frac{\delta(m_{k + 1}) * Pr(I_{k + 1} = 0 \mid I_{k} = 1, v_{k}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= Pr(I_{k + 1} = 0 \mid I_{k} = 1, v_{k}) * \int \Big[ \frac{\delta(m_{k + 1}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}\big) * \int \Big[ \frac{\delta(m_{k + 1}) * p(x_{k+1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}\big) * \frac{p(x_{k+1}=m_{k + 1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}=m_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})}
\end{align*}
$$

+ $I_{k + 1} = 1, I_{k} = 0$
$$
\begin{align*}
weights &= \int \Big[ \frac{p(x_{k+1} \mid x_{k}, I_{k} = 0, I_{k+1} = 1) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \int \Big[ \frac{\mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * \int \Big[ \frac{p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
\end{align*}
$$

+ $I_{k + 1} = 1, I_{k} = 1$
$$
\begin{align*}
weights &= \int \Big[ \frac{p(x_{k+1} \mid x_{k}, I_{k} = 1, I_{k+1} = 1) * Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= \int \Big[ \frac{\mathcal{N}(x_{k}, \sigma) * Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
\\ &= Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * \int \Big[ \frac{\mathcal{N}(x_{k}, \sigma) * p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
\end{align*}
$$


So we have:
+ $I_{k} = 0$
$$
\big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}\big) * \frac{p(x_{k+1}=m_{k + 1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}=m_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})}
\\ + \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * \int \Big[ \frac{p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
$$

+ $I_{k} = 1$
$$
\big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}\big) * \frac{p(x_{k+1}=m_{k + 1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}=m_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})}
\\ + Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * \int \Big[ \frac{\mathcal{N}(x_{k}, \hat{\sigma}) * p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
$$

### Algorithm

Set smoother posterior at time T to filter posterior at time T

$p(x_{T}, I_{T} \mid H_{1:T}) = p(x_{T}, I_{T} \mid H_{1:T})$

For each time step $k$:
1. Calculate Prior:
$
p(x_{k + 1}, I_{k + 1} = 0 \mid H_{1:k}) = \delta(m_{k+1}) * \big[\big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k})\big) * Pr(I_{k} = 0) + \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k})\big) * Pr(I_{k} = 1)\big]
$
$
p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k}) = \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * Pr(I_{k} = 0) + Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * \int \mathcal{N}(x_{k}, \hat{\sigma}) * p(x_{k}, I_{k} = 1 \mid H_{1:k}) dx_{k}
$
2. Compute Weights
    + $I_{k} = 0$
    $$
    \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}\big) * \frac{p(x_{k+1}=m_{k + 1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}=m_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})}
    \\ + \mathcal{U}(a, b) * Pr(I_{k + 1} = 1 \mid I_{k} = 0, v_{k}) * \int \Big[ \frac{p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
    $$

    + $I_{k} = 1$
    $$
    \big(1 - Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}\big) * \frac{p(x_{k+1}=m_{k + 1}, I_{k+1} = 0 \mid H_{1:T})}{p(x_{k + 1}=m_{k + 1}, I_{k + 1} = 0 \mid H_{1:k})}
    \\ + Pr(I_{k + 1} = 1 \mid I_{k} = 1, v_{k}) * \int \Big[ \frac{\mathcal{N}(x_{k}, \hat{\sigma}) * p(x_{k+1}, I_{k+1} = 1 \mid H_{1:T})}{p(x_{k + 1}, I_{k + 1} = 1 \mid H_{1:k})} \Big] dx_{k+1}
    $$
3. Backwards Update Posterior and Normalize
$$
p(x_{k}, I_{k} \mid H_{1:T}) = Normalize\Big(p(x_{k}, I_{k} \mid H_{1:k}) * weights\Big)
$$
